# Setup

### Model decisions

In [1]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
previous_loading = False
google_colab = False
moneyness = False
lags = 30
hyperparameter_search = False
training_size = 500000
val_size = 100000

In [3]:
if google_colab == True:
  from google.colab import drive
  drive.mount('/content/drive')

### Imports

In [4]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.math import multiply
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from math import log
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.metrics import mean_squared_error

In [5]:
# If running in colab, insert your wandb key here

#import config
#Erlend
wandb.login(key="3cae81eb56be3190be5bb48c571e69933071df69")
# Hjalmar
#wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load, split and normalize data

### Load data

In [6]:
if google_colab:
    import tensorflow as tf
    # Print info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "17fLS2n9NC_t2n6rzOpf3LeUJy2iEbdRY"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2010-2022_filtered.csv')  
    df_read = pd.read_csv('2010-2022_filtered.csv')
else:
    file = "../data/processed_data/2010-2022_filtered.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Unnamed: 0.1,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,R,Moneyness
0,0,0,2010-01-04,2010-01-07,207.490,1132.99,925.0,3,0.05,1.224854
1,1,1,2010-01-04,2010-01-07,182.500,1132.99,950.0,3,0.05,1.192621
2,2,2,2010-01-04,2010-01-07,157.500,1132.99,975.0,3,0.05,1.162041
3,3,3,2010-01-04,2010-01-07,132.600,1132.99,1000.0,3,0.05,1.132990
4,4,4,2010-01-04,2010-01-07,107.705,1132.99,1025.0,3,0.05,1.105356
...,...,...,...,...,...,...,...,...,...,...
11895723,13536141,13536141,2022-12-30,2024-12-20,362.600,3839.81,4300.0,721,4.41,0.892979
11895724,13536142,13536142,2022-12-30,2024-12-20,319.150,3839.81,4400.0,721,4.41,0.872684
11895725,13536143,13536143,2022-12-30,2024-12-20,279.000,3839.81,4500.0,721,4.41,0.853291
11895726,13536144,13536144,2022-12-30,2024-12-20,241.950,3839.81,4600.0,721,4.41,0.834741


### Create lags

In [7]:
df = df_read
del df_read

# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
#df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_last'].shift(i)

df = pd.merge(df, df_agg[['Quote_date'] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')

df = df.dropna()

### Format input data

In [8]:
# Format settings
max_timesteps = lags
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = [f'Underlying_{i}' for i in range (max_timesteps - 1, 0, -1)] + ['Underlying_last']

def create_rw_dataset(window_number = 0, df = None):
    '''Creates dataset for a single rolling window period offsett by the window number'''

    # Create train, validation and test set split points
    test_months = 6
    train_start = datetime(2014,1,1) + relativedelta(months=window_number * test_months)
    #val_start = train_start + relativedelta(months=5*12)
    #test_start = val_start + relativedelta(months = 1)
    test_start = train_start + relativedelta(months=4*12)
    test_end = test_start + relativedelta(months=test_months)
    train_start = str(train_start.date())
    #val_start = str(val_start.date())
    test_start = str(test_start.date())
    test_end = str(test_end.date())

        
    # Split train and validation data
    df_train = df[(df['Quote_date'] >= train_start) & (df['Quote_date'] < test_start)]
    #df_val = df[(df['Quote_date'] >= val_start) & (df['Quote_date'] < test_start)]
    df_test = df[(df['Quote_date'] >= test_start) & (df['Quote_date'] < test_end)]

    del df

    # Extract target values
    train_y = (df_train['Price'] / df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
    #val_y = (df_val['Price'] / df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()
    test_y = (df_test['Price'] / df_test['Strike']).to_numpy() if moneyness else df_test['Price'].to_numpy()

    # If usining moneyness, extract strike
    if moneyness:
        train_strike = df_train['Strike'].to_numpy()
    #    val_strike = df_val['Strike'].to_numpy()
        test_strike = df_test['Strike'].to_numpy()


    # Print earliest and latest date in every dataframe used
    print("--------------Dataframe dates--------------")
    print(f"Train: {df_train['Quote_date'].min()} - {df_train['Quote_date'].max()}")
    #print(f"Val: {df_val['Quote_date'].min()} - {df_val['Quote_date'].max()}")
    print(f"Test: {df_test['Quote_date'].min()} - {df_test['Quote_date'].max()}")
    print("-------------------------------------------")

    # Convert dataframes to numpy arrays
    train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
    #val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]
    test_x = [df_test[underlying_lags].to_numpy(), df_test[bs_vars].to_numpy()]

    del df_train
    #del df_val

    # Scale features based on training set
    underlying_scaler = MinMaxScaler()
    train_x[0] = underlying_scaler.fit_transform(train_x[0].flatten().reshape(-1, 1)).reshape(train_x[0].shape)
    #val_x[0] = underlying_scaler.transform(val_x[0])
    test_x[0] = underlying_scaler.transform(test_x[0].flatten().reshape(-1,1)).reshape(test_x[0].shape)

    bs_scaler = MinMaxScaler()
    train_x[1] = bs_scaler.fit_transform(train_x[1])
    #val_x[1] = bs_scaler.transform(val_x[1])
    test_x[1] = bs_scaler.transform(test_x[1])


    # Shuffle training set
    np.random.seed(0)
    shuffle = np.random.permutation(len(train_x[0]))
    train_x = [train_x[0][shuffle], train_x[1][shuffle]]
    train_y = train_y[shuffle]

    # Extract validation set
    val_x = [train_x[0][training_size:training_size+val_size], train_x[1][training_size:training_size+val_size]]
    val_y = train_y[training_size:training_size+val_size]

    # Extract training set
    train_x = [train_x[0][:training_size], train_x[1][:training_size]]
    train_y = train_y[:training_size]

    if moneyness:
        train_strike = train_strike[shuffle][:training_size]

    # Reshape data to fit LSTM
    train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps, 1), train_x[1]]
    val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]
    test_x = [test_x[0].reshape(len(test_x[0]), max_timesteps, 1), test_x[1]]

    print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
    print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')
    print(f'Test shape: {test_x[0].shape}, {test_x[1].shape}')

    if moneyness:
        return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike,
    #return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test
    return train_x, train_y, val_x, val_y, test_x, test_y, train_start, test_start, df_test

# Create the dataset for the first rolling window period
if moneyness:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset(df=df)
else:
    #train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df)
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, test_start, df_test = create_rw_dataset(df=df)

--------------Dataframe dates--------------
Train: 2014-01-03 - 2017-12-29
Test: 2018-01-02 - 2018-06-29
-------------------------------------------
Train shape: (500000, 30, 1), (500000, 4)
Val shape: (100000, 30, 1), (100000, 4)
Test shape: (599413, 30, 1), (599413, 4)


# Model construction

In [9]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.Interface_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Help functions

In [10]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [11]:
import gc
from tensorflow.keras import backend as k

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

## Creating trainer function

In [12]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None, previous_checkpoint_path = None, checkpoint_path = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        if previous_checkpoint_path and os.path.exists(previous_checkpoint_path + ".h5"):
            model = load_model(previous_checkpoint_path + ".h5")
        else:
            model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )
        
        # Check if the checkpoint folder exists
        if checkpoint_path and not os.path.exists(checkpoint_path):
            # Create the checkpoint folder if it does not exist
            os.makedirs(checkpoint_path)
        
        checkpoint = ModelCheckpoint(
            filepath=checkpoint_path + ".h5",
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=False
        )

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback, checkpoint, ClearMemory()] if checkpoint_path else [early_stopping, wandb_callback, ClearMemory()],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

# Hyperparameter search

In [13]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'bayes',
    'name': 'LSTM-MLP v4.0: fix nan issue',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'Interface_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [50, 100, 200, 400, 600]},
        'LSTM_timesteps': {
            'values': [10, 20, 40, 60, 90, 150]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0001)},
        'Lr_decay': {
            'distribution': 'log_uniform',
            'max': log(1), 'min': log(0.8)},        
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
#sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

if hyperparameter_search:
    wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 100)

# Rolling window

In [14]:
def calculate_error(predictions, original):
    m = MeanSquaredError()
    m.update_state(predictions, original)
    print("MSE:", m.result().numpy())
    m = RootMeanSquaredError()
    m.update_state(predictions, original)
    print("RMSE:", m.result().numpy())

class config_object:
    def __init__(self, config):
        self.LSTM_units = config['LSTM_units']
        self.Interface_units = config['Interface_units']
        self.MLP_units = config['MLP_units']
        self.LSTM_timesteps = config['LSTM_timesteps']
        self.LSTM_layers = config['LSTM_layers']
        self.MLP_layers = config['MLP_layers']
        self.Bn_momentum = config['Bn_momentum']
        self.Lr = config['Lr']
        self.Lr_decay = config['Lr_decay']
        self.Minibatch_size = config['Minibatch_size']
        self.Min_delta = config['Min_delta']
        self.Patience = config['Patience']
        self.Num_features = config['Num_features']
        self.Architecture = config['Architecture']
        

In [15]:
num_windows = 2 #84

config = {
    'LSTM_units': 4,
    'Interface_units': 16,
    'MLP_units': 400,
    'LSTM_timesteps': lags,
    'LSTM_layers': 6,
    'MLP_layers': 5,
    'Bn_momentum': 0.7,
    'Lr': 0.005,
    'Lr_decay': 0.9,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.E.1',
}

df_test_combined = pd.DataFrame()

checkpoint_time = datetime.now().strftime("%m-%d_%H-%M")

previous_checkpoint_path = None

for window in range(num_windows):
    if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset(df=df, window_number=window)
    else:
        #train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df, window_number=window)
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, test_start, df_test = create_rw_dataset(df=df, window_number=window)

    print('-------------------------------------------')
    print(f'Window {window + 1} of {num_windows}')
    #print("Training start: ", train_start, "Validation start: ", val_start, "Test start: ", test_start)
    print("Training start: ", train_start, "Test start: ", test_start)
    print("Loading previous checkpoint: ", previous_loading)
        
    if google_colab:
        checkpoint_path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/{checkpoint_time}/{train_start}/'
    else:
        checkpoint_path = f'./checkpoints/{checkpoint_time}/{train_start}/'

    #config['Dataset'] = f'{train_start} - {val_start} - {test_start}'
    config['Dataset'] = f'{train_start} - {test_start}'

    trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = config, project = 'Deep learning for option pricing - rolling windows',  previous_checkpoint_path = previous_checkpoint_path, checkpoint_path = checkpoint_path)
    c_model = load_model(checkpoint_path + ".h5")
    
    # Split test_x into five parts
    part_size = len(test_x[0]) // 5
    test_x_parts = []
    for i in range(5):
        subset_1 = test_x[0][i * part_size:(i + 1) * part_size]
        subset_2 = test_x[1][i * part_size:(i + 1) * part_size]
        test_x_parts.append((subset_1, subset_2))

    # Adjust the last part to include any remaining data
    test_x_parts[-1] = (test_x[0][(len(test_x_parts) - 1) * part_size:], test_x[1][(len(test_x_parts) - 1) * part_size:])

    # Make predictions for each part
    predictions_parts = [np.array(c_model(part)) for part in test_x_parts]

    # Combine the predictions
    predictions = np.concatenate(predictions_parts, axis=0)

    print(f'--- Predictions for test_start {test_start} ---')
    calculate_error(predictions, test_y)
    print('-------------------------------------------')
    
    df_test["Prediction"] = predictions
    df_test_combined = pd.concat([df_test_combined, df_test[["Quote_date", "Price", "Prediction"] + bs_vars]])

    if previous_loading:
       previous_checkpoint_path = checkpoint_path
       

print(f"--- All model predictions ---")
calculate_error(df_test_combined["Prediction"], df_test_combined["Price"])
print("-------------------------------------------")

if google_colab == False:
    predictions_path = './predictions/'
    if checkpoint_path and not os.path.exists(predictions_path):
        os.makedirs(predictions_path)
    df_test_combined.to_csv(f'{predictions_path}{datetime.now().strftime("%m-%d_%H-%M")}.csv')
else:
  path = '/content/drive/MyDrive/01. Masters Thesis - Shared/06. Predictions/41.csv'
  df_test_combined.to_csv(path)

--------------Dataframe dates--------------
Train: 2014-01-03 - 2017-12-29
Test: 2018-01-02 - 2018-06-29
-------------------------------------------
Train shape: (500000, 30, 1), (500000, 4)
Val shape: (100000, 30, 1), (100000, 4)
Test shape: (599413, 30, 1), (599413, 4)
-------------------------------------------
Window 1 of 2
Training start:  2013-01-01 Test start:  2018-01-01
Loading previous checkpoint:  False


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 16)           2016        ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                              

In [ ]:
# Define a function to calculate RMSE
def calculate_rmse(group):
    rmse_model_1 = np.sqrt(mean_squared_error(group["Price"], group["Model 1"]))
    return pd.Series({"RMSE_Model_1": rmse_model_1})

# Group data by Quote_date and calculate RMSE for each group
rmse_df = df.groupby("Quote_date").apply(calculate_rmse).reset_index()

# Plot the RMSE values for each model
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(data=rmse_df, x="Quote_date", y="RMSE_Model_1", label="Model", ax=ax)

# Set the interval for x-axis labels
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))

plt.xlabel("Quote_date")
plt.ylabel("RMSE")
plt.title("RMSE for Models")
plt.legend()
plt.xticks(rotation=45)
plt.show()
